Import libraries

In [9]:
import os

import pandas as pd

import numpy as np

from scipy import stats
import scipy.stats as sc
from scipy.cluster.hierarchy import linkage, leaves_list

import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from import_data import load_data

Import data

In [3]:
listVideo = load_data('../data_luc/video/')
    
nbVideos = len(listVideo)

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))

   configurationID  H264  no_8x8dct  no_asm  no_cabac  no_deblock  \
0                1  True       True   False     False        True   
1               10  True       True   False      True       False   
2              100  True       True   False     False        True   
3             1000  True       True   False      True       False   
4             1001  True      False   False     False        True   

   no_fast_pskip  no_mbtree  no_mixed_refs  no_weightb  rc_lookahead  ref  \
0           True      False           True        True            20    9   
1           True      False          False        True            40    9   
2          False       True           True       False            40    1   
3           True       True           True       False            40    9   
4          False      False           True       False            60    5   

      size  usertime  systemtime  elapsedtime  
0  1718492   22.9416      0.2784       3.1876  
1  1962957   26.8176      

## No transfer implementation

In [7]:
def no_transfer(target_id, nb_config_target_training,
                features, predictedVar, learning_algorithm):
    
    pct_test = 1-nb_config_target_training/listVideo[target_id].size
    
    random_state = np.random.randint(0,1000)

    # We define the target video, and split it into train-test
    target = listVideo[target_id]
    X_tgt = target[features]
    y_tgt = np.array(target[predictedVar], dtype=float)
    X_tgt_train, X_tgt_test, y_tgt_train, y_tgt_test = train_test_split(X_tgt, 
                                                                        y_tgt, 
                                                                        test_size=pct_test, 
                                                                        random_state=random_state)
    
    # The learning algorithm, training on the target video
    lf = learning_algorithm()
    lf.fit(X_tgt_train, y_tgt_train)
    y_tgt_pred_test = np.array(lf.predict(X_tgt_test)).reshape(-1,1)
    
    # We return the mean average percentage error 
    # between the real values of y_test from target 
    # and the predictions shifted 
    return min(mape(y_tgt_test, y_tgt_pred_test),1)


In [10]:
nbConfig = [16,32,64,128,256,320,448,640,960]
predictedVar = 'elapsedtime'
features = ['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
     'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']
learning_algorithm = RandomForestRegressor

res_no_transfer = pd.DataFrame({'id_video' : range(nbVideos)})

mape_no_transfer = np.ones(nbVideos)

nb_runs = 5
nb_conf = listVideo[0].size

for conf in nbConfig:
    
    pct_test = 1-conf/nb_conf
    
    for id_video in range(nbVideos):
        
        res_accuracy = []
        
        for i in range(nb_runs):
            
            res_accuracy.append(no_transfer(target_id=id_video,
                                            nb_config_target_training=conf,
                                            features=features, 
                                            predictedVar=predictedVar, 
                                            learning_algorithm=learning_algorithm))
            
        mape_no_transfer[id_video] = np.mean(res_accuracy)

    res_no_transfer[str(conf)] = mape_no_transfer

In [11]:
res = res_no_transfer.set_index('id_video')
res.to_csv("../data_luc/res/res_no_transfer_nb_conf_training.csv")
res

,16,32,64,128,256,320,448,640,960
id_video,,,,,,,,,
0,0.365396,0.291004,0.295874,0.288158,0.320095,0.321877,0.312874,0.329847,0.338852
1,0.310200,0.274831,0.246655,0.253265,0.264792,0.275877,0.280708,0.291336,0.299091
2,0.309788,0.204866,0.203701,0.219168,0.234847,0.228251,0.237970,0.247361,0.246105
3,0.320198,0.242020,0.272712,0.271745,0.271447,0.305870,0.292798,0.296194,0.302893
4,0.234140,0.215261,0.200537,0.226738,0.253417,0.252332,0.254327,0.252828,0.262700
5,0.260638,0.379124,0.232841,0.267321,0.270159,0.265127,0.288655,0.295965,0.297687
6,0.294299,0.316103,0.261092,0.266832,0.306138,0.301977,0.311578,0.318167,0.314954
7,0.263606,0.243475,0.245461,0.271686,0.278570,0.272964,0.288089,0.284698,0.286346
8,0.243851,0.243585,0.210515,0.229896,0.251614,0.254585,0.265652,0.262597,0.263756
